# Scripts de mosaicos

In [1]:
import ee
import geemap
import os
import folium

## Área de Estudio

In [143]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()

In [3]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

### Patagonia

In [4]:
work_dir = os.path.expanduser('../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')

## Funciones

In [33]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [34]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [35]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [36]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [37]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [38]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [39]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [40]:
def Cloud(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30))  

In [127]:
def Cloud50(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(50))  

In [41]:
def Cloud2(image):
    cloud = ee.Algorithms.Landsat.simpleCloudScore(image)
    scored = cloud.select(['cloud']).lte(20)
    return image.updateMask(scored) 

In [43]:
def Agua(image):
    return image.updateMask(ndwi.lte(-0.1))

In [16]:
def Ndvi1(image):
    return image.updateMask(ndvi.lt(0.2))

In [17]:
def Ndvi2(image):
    return image.updateMask(ndvi.gte(0.2))

## Generación de Colecciones

In [157]:
start_date = '2020-10-01'
end_date = '2021-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [158]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [159]:
coleccion = l8.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [160]:
Map.addLayer(l8, vizParams, 'false color composite')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

### Greenest

In [46]:
greenest = coleccion.qualityMosaic('NDVI')

In [161]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(greenest, visParams, 'comp mas verde')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [162]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [163]:
image_rgb = greenest.visualize(**{
    'bands' : ['B5','B4','B3'],
    'max':0.5
})

In [164]:
Map.addLayer(image_rgb, {}, 'test visualizacion')

#### Máscara

In [132]:
coleccion2 = coleccion.map(Cloud50)
coleccion2 = coleccion2.map(Agua)
greenest2 = coleccion2.qualityMosaic('NDVI')
map_sin_nubes = folium.Map(location = [-45, -73], zoom_start = 5)
map_sin_nubes.add_ee_layer(greenest2, vizParams, 'sin nubes')
display(map_sin_nubes)

#### Test

In [141]:
coleccion22 = coleccion.map(Cloud)
coleccion22 = coleccion22.map(Agua)

greenest22 = coleccion22.qualityMosaic('NDVI')

In [142]:
map_sin_nubes = folium.Map(location = [-45, -73], zoom_start = 7)
map_sin_nubes.add_ee_layer(greenest22, vizParams, 'sin nubes')
display(map_sin_nubes)

In [137]:
coleccion0 = coleccion.map(Cloud50)
coleccion0 = coleccion0.map(Agua)

greenest0 = coleccion0.qualityMosaic('NDVI')

In [138]:
map_sin_nubes = folium.Map(location = [-45, -73], zoom_start = 7)
map_sin_nubes.add_ee_layer(greenest0, vizParams, 'sin nubes')
display(map_sin_nubes)

In [67]:
greenest2 = greenest2.visualize(**{
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
})

In [165]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(greenest0, visParams, '2020-2021')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(bottom=11117.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['positio…

#### Temporada 2019-2020

In [166]:
start_date = '2019-10-01'
end_date = '2020-03-31'

landsat8_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [167]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [168]:
coleccion2020 = landsat8_2020.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [169]:
greenest2020 = coleccion2020.qualityMosaic('NDVI')

In [170]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(greenest2020, visParams, '2019-2020')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(bottom=11117.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['positio…

In [171]:
coleccion20 = coleccion2020.map(Cloud50)
coleccion20 = coleccion20.map(Agua)
greenest20 = coleccion20.qualityMosaic('NDVI')
map_sin_nubes = folium.Map(location = [-45, -73], zoom_start = 5)
map_sin_nubes.add_ee_layer(greenest20, vizParams, 'sin nubes')
display(map_sin_nubes)

In [182]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(greenest20, visParams, '2019-2020')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(bottom=11117.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['positio…

#### Temporada 2018-2019

In [172]:
start_date = '2018-10-01'
end_date = '2019-03-31'

landsat8_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [173]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [174]:
coleccion2019 = landsat8_2019.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [175]:
greenest2019 = coleccion2019.qualityMosaic('NDVI')

In [176]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(greenest2019, visParams, '2018-2019')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(bottom=11117.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['positio…

In [177]:
coleccion19 = coleccion2019.map(Cloud50)
coleccion19 = coleccion19.map(Agua)
greenest19 = coleccion19.qualityMosaic('NDVI')
map_sin_nubes19 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sin_nubes19.add_ee_layer(greenest19, vizParams, 'sin nubes')
display(map_sin_nubes19)

In [183]:
visParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(greenest19, visParams, '2018-2019')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(bottom=22367.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['positio…

### Agua

In [178]:
start_date_agua = '2020-11-01'
end_date_agua = '2021-03-31'

l8_agua = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date_agua, end_date_agua)


In [179]:
coleccion_agua = l8_agua.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [180]:
aquast = coleccion_agua.qualityMosaic('NDWI')

In [181]:
visParams = {
  'bands': ['B4', 'B3', 'B1'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(aquast, visParams, 'comp mas agua')
Map.centerObject(patagonia_grande, 6)
Map

Map(bottom=1851.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position…

In [84]:
ndwi = greenest.select('NDWI')
ndwi_viz = {'min': -1, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.addLayer(ndwi, ndwi_viz, 'NDWI')

In [85]:
ndwi_rgb = ndwi.visualize(**{
    'min': 0.5,
    'max':1,
    'palette': ['00FFFF', '0000FF']
})

#### Máscara

In [86]:
ndwi_mask = ndwi.updateMask(ndwi.gte(-0.1)) # Máscara partes sin agua, NDWI < -0.1

In [87]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask.add_ee_layer(ndwi_mask, ndwi_viz, 'NDWI sin tierra')
display(map_ndwi_mask)

In [88]:
coleccion3 = coleccion.map(Cloud)
coleccion3 = coleccion3.map(Ndvi1)
aquast2 = coleccion3.qualityMosaic('NDWI')

In [89]:
map_sin_tierra = folium.Map(location = [-45, -73], zoom_start = 7)
map_sin_tierra.add_ee_layer(aquast2, vizParams, 'sin tierra')
display(map_sin_tierra)

In [90]:
coleccion4 = coleccion.map(Cloud)
coleccion4 = coleccion4.map(Agua)
# coleccion4 = coleccion4.map(Ndvi2)

greenest3 = coleccion4.qualityMosaic('NDVI')

In [91]:
map_sin_nubes = folium.Map(location = [-45, -73], zoom_start = 7)
map_sin_nubes.add_ee_layer(greenest3, vizParams, 'sin nubes')
display(map_sin_nubes)

### Tierra

In [92]:
ndgi = greenest.select('NDGI')
ndgi_viz = {'min': -1, 'max': 1, 'palette': ['FF00FF', 'FFFF00']}
Map.addLayer(ndgi, ndgi_viz, 'NDGI')

In [93]:
start_date = '2020-01-01'
end_date = '2021-03-31'

l8tierra = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [94]:
coleccion_tierra = l8tierra.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [95]:
tierrast = coleccion_tierra.qualityMosaic('NDGI')

In [96]:
Map.addLayer(tierrast, vizParams, 'test tierra')
Map

Map(bottom=11117.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['positio…

## Mosaicos

In [144]:
mosaic = ee.ImageCollection([greenest,aquast]).mosaic()

In [145]:
Map3.addLayer(mosaic, vizParams, 'Mosaico')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [146]:
map_mosaic = folium.Map(location = [-45, -73], zoom_start = 7)
map_mosaic.add_ee_layer(mosaic, vizParams, 'mosaic')
display(map_mosaic)

In [147]:
mosaic2 = ee.ImageCollection([aquast2, greenest3]).mosaic()

In [148]:
mosaic2.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'NDWI',
 'NDGI',
 'date']

In [149]:
Map3.addLayer(mosaic2, vizParams, 'Mosaico2')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [150]:
mosaic3 = ee.ImageCollection([greenest20,mosaic2]).mosaic()

In [151]:
Map3.addLayer(mosaic3, vizParams, 'Mosaico3')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [152]:
mosaic4 = ee.ImageCollection([greenest19,mosaic3]).mosaic()

In [153]:
Map3.addLayer(mosaic4, vizParams, 'Mosaico4')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [154]:
# coleccion_2020 = coleccion2020.map(Cloud)
coleccion_2020 = coleccion2020.map(Agua)

greenest_20 = coleccion_2020.qualityMosaic('NDVI')

In [155]:
mosaico_green = ee.ImageCollection([greenest_20,mosaic3]).mosaic()

In [156]:
Map3.addLayer(mosaico_green, vizParams, 'Mosaico_verde')
Map3

Map(bottom=1851.0, center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(Togg…

In [110]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(bottom=11117.0, center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['positio…

## Clasificación Supervisada

In [111]:
work_dir = os.path.expanduser('../Data/shapes/')

val_shp= os.path.join(work_dir, 'validacion.shp')
merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [112]:
merged = geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [113]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de Validación')

In [114]:
dataset = mosaic4.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [115]:
datasetval = mosaic4.sampleRegions(**{
    'collection': val,
    'properties': ['ID'],
    'scale': 10,
})

In [116]:
print(dataset.size().getInfo())

20295


In [117]:
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'NDWI',
 'NDGI']

label = 'ID'

In [118]:
randomforest = ee.Classifier.smileRandomForest(100).train(dataset, label, bands)
resultRF = mosaic4.select(bands).classify(randomforest)

In [119]:

class_values1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

class_palette1 = ['22c129', # 1
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [120]:
resultRF = resultRF.set('classification_class_values', class_values1)
resultRF = resultRF.set('classification_class_palette', class_palette1)

In [121]:
Map.addLayer(resultRF, {}, 'Land cover clasificado RF 100' )
legend_dict = {
    '01 Achaparrado': '22c129',
    '02 Agrícola': 'f0ff00',
    '03 Agua': '0c40f1',
    '04 Arenal o Roca': '685205',
    '05 Bosque Degradado': '0d4404',
    '06 Cuncunas': 'e82d13',
    '07 Estepa': 'f99504',
    '08 Humedal': '17d4c6',
    '09 Mallin': '1c6330',
    '10 Matorral abierto': '24ff00',
    '11 Matorral denso': '2de50f',
    '12 Plantaciones': 'ccba7d',
    '13 Praderas': 'dec61a',
    '14 Primario': '28d909',
    '15 Ríos': '03c2ff',
    '16 Secundario': '4e8c44',
    '17 Sombra topo': 'fdfdfd',
    '18 Suelo desnudo': 'c0c233'
}
Map.add_legend(legend_title="Random Forest", legend_dict=legend_dict)
Map

Map(bottom=11363.0, center=[-53.21261218994158, -69.40063476562501], controls=(WidgetControl(options=['positio…